[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Etiquetado_openai.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [1]:
!pip install -q PyPDF2 python-docx openpyxl python-pptx xlrd transformers office365-rest-python-client msal requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 9.0 MB/s eta 0:00:00


In [41]:
import os
import io
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader
import docx
import openpyxl
from pptx import Presentation
import xlrd
from transformers import pipeline
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import os
import requests
import msal
import csv
import getpass
from openai import OpenAI
from transformers import pipeline
import time
import openai
import requests

In [25]:
api_key = getpass.getpass("Introduce tu OpenAI API Key: ")
client = OpenAI(api_key=api_key)
token = getpass.getpass("Introduce tu GitHub token: ")

Introduce tu OpenAI API Key: ··········
Introduce tu GitHub token: ··········


In [26]:
!apt-get install git -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


# Configuración

In [27]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"  # tu nuevo Client ID
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]

ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
classifier = pipeline("zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli")
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"

Device set to use cuda:0


In [28]:
!git config --global user.email "marcomendieta08@gmail.com"
!git config --global user.name "marcomendieta08"
!git clone https://github.com/pugapatricia/gestion-documentaria-para-pymes.git

Cloning into 'gestion-documentaria-para-pymes'...
remote: Enumerating objects: 276, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 276 (delta 39), reused 16 (delta 8), pack-reused 208 (from 1)
Receiving objects: 100% (276/276), 6.47 MiB | 13.44 MiB/s, done.
Resolving deltas: 100% (145/145), done.


# Conección con OneDrive


In [6]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()

To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code T9RMTAVR to authenticate.


#Funciones

In [7]:
ticketsTXT = "https://raw.githubusercontent.com/pugapatricia/gestion-documentaria-para-pymes/refs/heads/main/etiquetado/tickers.txt"
response = requests.get(ticketsTXT)
etiquetas = response.text.strip().split(", ")
print(etiquetas)



['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad', 'arqueología', 'financiero', 'contaduría', 'restitución', 'materias', 'regresión', 'extranjero', 'renta', 'análisis', 'puente', 'servicios', 'inquilino', 'optativa', 'arrendador', 'seguridad', 'notificaciones', 'vigilancia', 'depreciación', 'empresa', 'geografía', 'monográficos', 'estancia', 'balance', 'divulgación', 'autorización', 'master', 'madrid', 'interpretación', 'ingresos', 'métodos', 'fuentes', 'idiomas', 'silvopastoril', 'extranjería', 'receptor', 'proyecto', 'interés', 'apalancamiento', 'propiedad', 'incumplimiento', 'estudios', 'nie', 'datascience', 'complutense', 'mediación', 'firma', 'liquidez', 'valoracion', 'desalojo', 'compraventa', 'alquiler', 'sociales', 'jarama', 'partes', 'contrato', 'tasas', 'humanidades', 'estudiantes', 'activos', 'actividad', 'pasivos', 'financieros', 'registro', 'divulgador', 'cesión', 'poesía', 'temporada', 'documentación', 'rentabilidad', 'comisiones', 'flujodecaja', 'patri

Lector de documentos

In [8]:
def leer_pdf(contenido, limite_palabras):
    texto = ""
    reader = PdfReader(io.BytesIO(contenido))
    for page in reader.pages:
        if page.extract_text():
            texto += page.extract_text() + "\n"
            if len(texto.split()) >= limite_palabras:
                break
    return texto


def leer_docx(contenido, limite_palabras):
    texto = ""
    doc = docx.Document(io.BytesIO(contenido))
    for p in doc.paragraphs:
        if p.text.strip():
            texto += p.text + "\n"
            if len(texto.split()) >= limite_palabras:
                break
    return texto


def leer_excel(contenido, limite_palabras):
    texto = ""
    wb = openpyxl.load_workbook(io.BytesIO(contenido), data_only=True, read_only=True)
    for sheet in wb.worksheets:
        for row in sheet.iter_rows(values_only=True):
            texto += " ".join([str(cell) for cell in row if cell]) + "\n"
            if len(texto.split()) >= limite_palabras:
                break
        if len(texto.split()) >= limite_palabras:
            break
    return texto


def leer_xls(contenido, limite_palabras):
    texto = ""
    temp_file = "temp.xls"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    wb = xlrd.open_workbook(temp_file)
    for sheet in wb.sheets():
        for row_idx in range(sheet.nrows):
            row = sheet.row_values(row_idx)
            texto += " ".join([str(cell) for cell in row if cell]) + "\n"
            if len(texto.split()) >= limite_palabras:
                break
        if len(texto.split()) >= limite_palabras:
            break
    os.remove(temp_file)
    return texto


def leer_pptx(contenido, limite_palabras):
    texto = ""
    temp_file = "temp.pptx"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    prs = Presentation(temp_file)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                texto += shape.text + "\n"
                if len(texto.split()) >= limite_palabras:
                    break
        if len(texto.split()) >= limite_palabras:
            break
    os.remove(temp_file)
    return texto


def leer_txt_csv(contenido, limite_palabras):
    texto = contenido.decode("utf-8", errors="ignore")
    return " ".join(texto.split()[:limite_palabras])

def leer_archivo(nombre, contenido, limite_palabras=None):
    ext = nombre.split(".")[-1].lower()
    if ext == "pdf":
        return leer_pdf(contenido, limite_palabras)
    elif ext == "docx":
        return leer_docx(contenido, limite_palabras)
    elif ext == "xlsx":
        return leer_excel(contenido, limite_palabras)
    elif ext == "xls":
        return leer_xls(contenido, limite_palabras)
    elif ext == "pptx":
        return leer_pptx(contenido, limite_palabras)
    elif ext in {"txt", "csv"}:
        return leer_txt_csv(contenido, limite_palabras)
    return ""

Clasificador Hugging Face Zero-Shot


In [40]:
def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = texto.lower()
    texto = re.sub(r"[\r\n\t]+", " ", texto)
    texto = re.sub(r"[^a-záéíóúüñ0-9\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

def etiquetar_texto(texto, umbral=0.3, min_etiquetas=5, max_etiquetas=10):
    res = classifier(texto, candidate_labels=etiquetas, multi_label=True)
    etiquetas_filtradas = [label for label, score in zip(res["labels"], res["scores"]) if score >= umbral]
    if len(etiquetas_filtradas) < min_etiquetas:

        etiquetas_ordenadas = [label for _, label in sorted(zip(res["scores"], res["labels"]), reverse=True)]
        for label in etiquetas_ordenadas:
            if label not in etiquetas_filtradas:
                etiquetas_filtradas.append(label)
            if len(etiquetas_filtradas) >= min_etiquetas:
                break
    etiquetas_filtradas = etiquetas_filtradas[:max_etiquetas]

    return etiquetas_filtradas


In [42]:
def generar_descripcion(texto):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # o "gpt-4-turbo" si quieres más calidad
            messages=[
                {"role": "system", "content": "Eres un asistente que resume documentos brevemente."},
                {"role": "user", "content": f"Resume en 2 líneas el siguiente texto:\n\n{texto}"}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generando descripción: {e}"


# Leer archivos de OneDrive

In [43]:
limite_palabras = 2000
response = requests.get(url, headers=headers)
data = response.json()

resultados = {}
descripciones = {}

for item in data.get("value", []):  # Procesar todos los archivos
    nombre = item["name"]
    if not any(nombre.lower().endswith(ext) for ext in ext_permitidas):
        continue
    download_url = item["@microsoft.graph.downloadUrl"]
    file_bytes = requests.get(download_url).content
    texto = leer_archivo(nombre, file_bytes, limite_palabras)

    # Mantienes tu lógica actual
    resultados[nombre] = etiquetar_texto(texto) if texto else []

    # Agregas la descripción nueva
    descripciones[nombre] = describir_texto(texto) if texto else "No se pudo leer el archivo"

# Mostrar resultados
for nombre, etiquetas_detectadas in resultados.items():
    print(f"{nombre} → {etiquetas_detectadas}")
    print(f"Descripción: {descripciones[nombre]}\n")


11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf → ['registro', 'propiedad', 'arrendador', 'indicadores', 'empaques', 'propiedadintelectual', 'inventarios', 'jurisdicción', 'temporada', 'divulgación']
Descripción: Error generando descripción: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


2016-admitidos_Segundo ciclo- Cursos monográficos.xls → ['historia', 'confidencialidad', 'arte', 'medioambiente', 'divulgador']
Descripción: Error generando descripción: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0

In [14]:
limite_palabras = 2000
response = requests.get(url, headers=headers)
data = response.json()

resultados = {}
for item in data.get("value", []):  # Procesar todos los archivos
    nombre = item["name"]
    if not any(nombre.lower().endswith(ext) for ext in ext_permitidas):
        continue
    download_url = item["@microsoft.graph.downloadUrl"]
    file_bytes = requests.get(download_url).content
    texto = leer_archivo(nombre, file_bytes, limite_palabras)
    resultados[nombre] = etiquetar_texto(texto) if texto else []

for nombre, etiquetas_detectadas in resultados.items():
    print(f"{nombre} → {etiquetas_detectadas}")

11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf → ['registro', 'propiedad', 'arrendador', 'indicadores', 'empaques', 'propiedadintelectual', 'inventarios', 'jurisdicción', 'temporada', 'divulgación']
2016-admitidos_Segundo ciclo- Cursos monográficos.xls → ['historia', 'confidencialidad', 'arte', 'medioambiente', 'divulgador']
2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx → ['seguro', 'seguridad', 'obligaciones', 'solicitud', 'jurisdicción', 'ingresos', 'reserva', 'cursos', 'contaduría', 'regresión']
660d1bfb7c43622a597a4000_Non-disclosure agreement nda template contract.pdf → ['solicitud', 'propiedadintelectual', 'balance', 'incumplimiento', 'restitución', 'endeudamiento', 'indicadores', 'vigilancia', 'datascience', 'arrendamiento']
Acuerdo_no_Divulgacion_Unilateral_UE.pdf → ['cesión', 'medioambiente', 'arrendatario', 'registro', 'arte', 'confidencialidad', 'divulgación', 'silvopastoril', 'tir', 'cultivo']
Acuerdo-de-Confidencialidad-O

In [ ]:
def describir_texto(texto: str) -> str:
    """Genera una breve descripción del contenido usando OpenAI."""
    if not texto.strip():
        return "Sin contenido legible"

    prompt = f"""
    Resume brevemente de qué trata el siguiente texto en máximo 3 líneas.
    Sé claro y directo, sin inventar información.

    Texto:
    {texto[:4000]}  # límite de caracteres para evitar sobrecarga
    """

    try:
        respuesta = openai.ChatCompletion.create(
            model="gpt-5-mini",  # puedes usar gpt-4o, gpt-4o-mini o gpt-3.5-turbo
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150
        )
        descripcion = respuesta.choices[0].message.content.strip()
        return descripcion
    except Exception as e:
        return f"Error generando descripción: {e}"

# === PROCESAMIENTO ===
limite_palabras = 2000
response = requests.get(url, headers=headers)
data = response.json()

resultados = {}
descripciones = {}

for item in data.get("value", []):
    nombre = item["name"]
    if not any(nombre.lower().endswith(ext) for ext in ext_permitidas):
        continue

    download_url = item["@microsoft.graph.downloadUrl"]
    file_bytes = requests.get(download_url).content
    texto = leer_archivo(nombre, file_bytes, limite_palabras)

    if texto:
        resultados[nombre] = etiquetar_texto(texto)
        descripciones[nombre] = describir_texto(texto)
    else:
        resultados[nombre] = []
        descripciones[nombre] = "No se pudo leer el archivo"

# === MOSTRAR RESULTADOS ===
for nombre in resultados:
    print(f"📄 {nombre}")
    print(f"   🏷️ Etiquetas: {resultados[nombre]}")
    print(f"   📝 Descripción: {descripciones[nombre]}")
    print("-" * 60)


#Guardar resultados

In [20]:
%cd /content/gestion-documentaria-para-pymes

/content/gestion-documentaria-para-pymes


In [37]:

os.makedirs("etiquetado", exist_ok=True)
csv_path = "etiquetado/etiquetas_onedrive.csv"
with open(csv_path, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Archivo", "Etiquetas"])
    for nombre, etiquetas_detectadas in resultados.items():
        etiquetas_str = ", ".join(etiquetas_detectadas) if etiquetas_detectadas else "Sin etiquetas"
        writer.writerow([nombre, etiquetas_str])
print(f"✅ Resultados guardados en {csv_path}")


✅ Resultados guardados en etiquetado/etiquetas_onedrive.csv


In [38]:
!ls etiquetado/

Etiquetado_huggingface.ipynb  Generador_etiquetasHF.ipynb
Etiquetado_openai.ipynb       Generador_etiquetas.ipynb
etiquetas_onedrive.csv	      tickers.txt
flujo-de-proceso.md	      UnionEtiquetado_open.ipynb


In [39]:
!git add etiquetado/etiquetas_onedrive.csv
!git diff --cached --quiet || git commit -m "Actualizar etiquetas_onedrive.csv desde Colab"
!git pull https://{token}@github.com/pugapatricia/gestion-documentaria-para-pymes.git main
!git push https://{token}@github.com/pugapatricia/gestion-documentaria-para-pymes.git main

[main 172cda4] Actualizar etiquetas_onedrive.csv desde Colab
 1 file changed, 26 insertions(+)
 create mode 100644 etiquetado/etiquetas_onedrive.csv
From https://github.com/pugapatricia/gestion-documentaria-para-pymes
 * branch            main       -> FETCH_HEAD
Already up to date.
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 358 bytes | 358.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/pugapatricia/gestion-documentaria-para-pymes.git
   5ff777a..172cda4  main -> main


## Codigo para cargar en la etiqueta de descripcion en onedrive

In [ ]:
"""
# ============================
# Configuración
# ============================
json_path = "etiquetas_onedrive.json"
headers = {"Authorization": f"Bearer {access_token}"}

# Carpeta objetivo en OneDrive
carpeta_objetivo = "Etiquetados"

# ============================
# Cargar JSON de etiquetas
# ============================
with open(json_path, "r", encoding="utf-8") as f:
    etiquetas_data = json.load(f)

# ============================
# Aplicar etiquetas en la descripción
# ============================
for archivo, etiquetas in etiquetas_data.items():
    if not etiquetas:
        continue

    # Buscar archivo en OneDrive
    url_file = f"https://graph.microsoft.com/v1.0/me/drive/root:/{carpeta_objetivo}/{archivo}"
    resp_file = requests.get(url_file, headers=headers)

    if resp_file.status_code != 200:
        print(f"⚠️ No se encontró {archivo} en OneDrive")
        continue

    file_id = resp_file.json()["id"]

    # Guardamos etiquetas en el campo "description"
    url_update = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}"
    payload = {"description": ", ".join(etiquetas)}

    response_update = requests.patch(
        url_update,
        headers={**headers, "Content-Type": "application/json"},
        json=payload
    )

    if response_update.status_code in [200, 204]:
        print(f"✅ Etiquetas {etiquetas} aplicadas a {archivo} en la descripción")
    else:
        print(f"⚠️ Error al actualizar {archivo}: {response_update.text}")
"""